In [127]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import helper
from sklearn import metrics


In [129]:
# Загрузить данные из файлов
matches706_team_picks = pd.read_csv('table with features teams, heroes & kills (all team) 7.06.csv').drop(['Unnamed: 0'], axis=1)
# matches707_team_picks = pd.read_csv('table with carry or support in 7.07 to 29.12.17.csv', index_col=0)

# оставить только героев:
matches706_heroes = matches706_team_picks.loc[:, :'Pangolier']
matches706_heroes['radiant_win'] = matches706_team_picks['radiant_win']
matches706_heroes['kills'] = matches706_team_picks['kills']

#чтобы не переписывать код:
matches706_team_picks = matches706_heroes

# # Соединить даныне
# matches_team_picks = matches706_team_picks.append(matches707_team_picks).reset_index().drop(['index'], axis=1)

#Разделение данных без перемешивания на traning & test для каждого патча и для соединенных данных
X_706, y_706 = np.split(matches706_team_picks, [len(matches706_team_picks) - int(len(matches706_team_picks)*0.3)]) 
# X_707, y_707 = np.split(matches707_team_picks, [len(matches707_team_picks) - int(len(matches707_team_picks)*0.3)]) 
# X, y = np.split(matches_team_picks, [len(matches_team_picks) - 400]) 

# X_train = X.drop(['radiant_win'], axis=1)
# y_train = X['radiant_win']

# X_test = y.drop(['radiant_win'], axis=1)
# y_test = y['radiant_win']

X_train_706 = X_706.drop(['radiant_win', 'kills'], axis=1)
y_train_706_kills = X_706['kills']
y_train_706 = X_706['radiant_win']

X_test_706 = y_706.drop(['radiant_win','kills'], axis=1)
y_test_706_kills = y_706['kills']
y_test_706 = y_706['radiant_win']

# X_train_707 = X_707.drop(['radiant_win'], axis=1)
# y_train_707 = X_707['radiant_win']

# X_test_707 = y_707.drop(['radiant_win'], axis=1)
# y_test_707 = y_707['radiant_win']


In [183]:
X_train_706_ = X_train_706#[:200]
#y_train_706_ = y_train_706#[:200]
y_train_706_ = y_train_706_kills#[:200]

X_test_706_ = X_test_706[211:]
#y_test_706_ = y_test_706#[201:261]
y_test_706_ = y_test_706_kills[211:]
len

In [185]:
# дерево 
tree = DecisionTreeClassifier()
tree.fit(X_train_706_, y_train_706_)
print (tree.score(X_train_706_, y_train_706_))
print (tree.score(X_test_706_, y_test_706_))
print ('------------------------------------------------------')
tree_reg = DecisionTreeClassifier()
tree_reg.fit(X_train_706_, y_train_706_kills)
print (tree.score(X_train_706_, y_train_706_kills))
# print (tree.predict(X_test_706_))
# print (np.array(y_test_706_kills))
a = []
for i in range(len(y_test_706_)):
    if tree_reg.predict(X_test_706_)[i] < 46.5 and np.array(y_test_706_)[i] < 46.5:
        a.append(1)
        continue
    if tree_reg.predict(X_test_706_)[i] > 46.5 and np.array(y_test_706_)[i] > 46.5:
        a.append(1)
        continue
    else:
        a.append(0)
        continue
print (a.count(1) / len(a))
print (len(y_test_706_kills))

1.0
0.0149253731343
------------------------------------------------------
1.0
0.5223880597014925
479


In [186]:
# Случайный лес с параметрами которые оказались лучшими на обучении патча7,06
forest_last = RandomForestClassifier(random_state=42)
forest_last.fit(X_train_706_, y_train_706_)
print (forest_last.score(X_train_706_, y_train_706_))
print (forest_last.score(X_test_706_, y_test_706_))
print ('------------------------------------------------------')
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train_706_, y_train_706_kills)

a = []
for i in range(len(y_test_706_)):
    if forest_reg.predict(X_test_706_)[i] < 46.5 and np.array(y_test_706_)[i] < 46.5:
        a.append(1)
        continue
    if forest_reg.predict(X_test_706_)[i] > 46.5 and np.array(y_test_706_)[i] > 46.5:
        a.append(1)
        continue
    else:
        a.append(0)
        continue
print (a.count(1) / len(a))
print (len(y_test_706_kills))

def predict_log_proba (pred_log_prob):
        arr_pred = []
        for  x in (pred_log_prob):
            if (abs(x[0]) > abs(x[1])):
                arr_pred.append(0)
            else:
                arr_pred.append(1)
        return (np.array(arr_pred))

def predict_proba (pred_prob, perc):
        arr_pred = []
        arr_ind = []
        for i, x in enumerate(pred_prob):
            if (x[0]>=perc):
                arr_pred.append(0)
                arr_ind.append(i)
                continue
            if(x[1]>=perc):
                arr_pred.append(1)
                arr_ind.append(i)
                continue
            else:
                arr_pred.append('X')
                continue
        return (np.array(arr_pred), arr_ind)

0.997321428571
0.0186567164179
------------------------------------------------------
0.5485074626865671
479


In [188]:
# Градиентный бустинг
# gbr = GradientBoostingClassifier(random_state=42)
# gbr.fit(X_train_706_, y_train_706_)
# print (gbr.score(X_train_706_, y_train_706_))
# print (gbr.score(X_test_706_, y_test_706_))
print ('------------------------------------------------------')
gbr_reg = GradientBoostingRegressor(n_estimators=200)
gbr_reg.fit(X_train_706_, y_train_706_kills)

a = []
for i in range(len(y_test_706_)):
    if gbr_reg.predict(X_test_706_)[i] < 46.5 and np.array(y_test_706_)[i] < 46.5:
        a.append(1)
        continue
    if gbr_reg.predict(X_test_706_)[i] > 46.5 and np.array(y_test_706_)[i] > 46.5:
        a.append(1)
        continue
    else:
        a.append(0)
        continue
print (a.count(1) / len(a))
print (len(y_test_706_kills))

------------------------------------------------------
0.582089552238806
479


In [190]:
print (gbr_reg.predict(X_test_706).std())
print (np.array(y_test_706_).std())

6.46021832479
15.6477407471


In [105]:
prob = gbr.predict_log_proba(X_test_706_)
predict = predict_log_proba(prob)
print (predict)
pred_prob= gbr.predict_proba(X_test_706_)
#print (pred_prob)
#predict, pred_ind = predict_proba(pred_prob, 0.8)
# print (predict)
# for i  in pred_ind:
#     print (np.array(y_test_706_)[i])
print (np.array(y_test_706_))
print (metrics.accuracy_score(np.array(y_test_706_), predict))

[0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0]
[0 1 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0]
0.483333333333


In [56]:
# создание большого кол-ва лесов с разными параметрами и запись Score в массив
from sklearn.ensemble import RandomForestClassifier
all_parametrs=[]
for n in range(50, 500, 25):
    for depth in range(50, 200, 25):
        for features in range(30, 190, 30):
            parametrs=[]
            forest_optim_param = RandomForestClassifier(n_estimators=n, random_state=42, 
                                                        max_depth=depth, max_features=features)
            forest_optim_param.fit(X_train_706_, y_train_706_)
            if (forest_optim_param.score(X_test_706_, y_test_706_) > 0.62):
                parametrs.append(n)
                parametrs.append(depth)
                parametrs.append(features)
                parametrs.append(forest_optim_param.score(X_train_706_, y_train_706_))
                parametrs.append(forest_optim_param.score(X_test_706_, y_test_706_))
                all_parametrs.append(parametrs)
    print (n)

50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475


In [57]:
all_parametrs

[]